# Plate Analysis Interactive V4.2

**by Conrad Hall** <br><br>

This program automates the analysis of a screen.<br><br>
The 5 parameters followed in this screen were:
-  ValidObjectCount (Equivalent to CellCount)
-  MEAN_TargetTotalIntenCh2
-  MEAN_ObjectAvgIntenCh1
-  MEAN_ObjectAreaCh1
-  %HIGH_TargetTotalIntenCh2 (Equivalent to %HighPFF)

Plotly is used to generate plots to aid in visualization of the data. 
The are also sliders available to threshold the data generating a candidate list that can be exported.

In [ ]:
import glob
import re
import string
import xlsxwriter
from operator import itemgetter
import pandas as pd
from collections import OrderedDict
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from ipywidgets import fixed
from IPython.display import display, Markdown, display_markdown, HTML
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools

init_notebook_mode(connected=True)



%matplotlib notebook

***
### Import Data
Imports all csv files in a hardcoded path then places the data into dataframes as part of a dictionary. The plate names are also pulled from the filename and stored.

In [ ]:
ctr_cmpd = 'DMSO'

# #Uncomment below (using command Ctrl+/) to enable user selection of the control compound used in the screen.

# #This module asks for the name of the control compound used in the screen. It will primarily be DMSO but there may be other controls potentially ethanol or water. This will handle those cases
# display_markdown('#### Is DMSO the control compound for this screen?',raw=True)
# button_DMSO_yes = widgets.Button(description="YES")
# button_DMSO_no = widgets.Button(description="NO")
# selected_compound = widgets.Output(layout={'border': '1px solid black'})
# button_save = widgets.Button(description="save")
# text_input = widgets.Text()
# display(button_DMSO_yes, button_DMSO_no, selected_compound)

# def on_yes_clicked(b):
#     global ctr_cmpd
#     ctr_cmpd = 'DMSO'
#     selected_compound.clear_output()
#     with selected_compound:
#         display_markdown('#### Control compound: DMSO',raw=True)

# def on_no_clicked(b):
#     display_markdown('#### Enter name of control compound:',raw=True)
#     display(text_input, button_save)
#     text_input.observe(on_value_change, names='value')
#     button_save.on_click(on_save_clicked)
    
# def on_save_clicked(b):
#     text_input.close()
#     button_save.close()

# def on_value_change(change):
#     global ctr_cmpd
#     ctr_cmpd = change['new']
#     selected_compound.clear_output()
#     with selected_compound:
#         display_markdown('#### Control compound:'+change['new'],raw=True)

# button_DMSO_yes.on_click(on_yes_clicked)
# button_DMSO_no.on_click(on_no_clicked)

In [ ]:
# Load Feature Data from CSV into a DataFrame
path = r'MoleculeData'
filenames = glob.glob(path + '/*.csv')
i=0
df_dict = {}
plate_name = []
regex = r'^.*[\/\\](\D+)(\d+|\d+\D+)\.\w+' # Regex finds two groups from the file name.First is the library name and second is the plate name
for filename in filenames:
    df = pd.read_csv(filename, index_col=0)
    plate_name.append(re.findall(regex,filename)[0][1])
    df_dict[i]= df
    i+=1
    
# Load Compound List from CSV into a DataFrame
drugpath = r'MoleculeData/drug_list'
drugfilename = glob.glob(drugpath + '/*.csv')
drugdf = pd.read_csv(drugfilename[0], encoding='cp1252')

#Rearrange Compound DataFrame to match Feature DataFrame
well_list = drugdf["well"].tolist()
drug_regex = r'^(\w)(\d+)'
ID = []
for entry in well_list:
    row = re.findall(drug_regex,entry)[0][0]
    col = re.findall(drug_regex,entry)[0][1]
    ID.append(100*(ord(row)-64) + int(col))

se = pd.Series(ID)
drugdf['WellID'] = se.values
drugdf = drugdf.set_index('WellID')
drugdf = drugdf.drop(columns='well')

# Merge Compound DataFrame into the Feature DataFrame
i=0
for name in plate_name:
    plateregex = r'^\d+'
    df1 = drugdf.loc[(drugdf['plate #'] == int(re.findall(plateregex,name)[0])) & drugdf['molecule']]
    df_dict[i] = pd.concat([df1['molecule'],df_dict[i]],axis=1, sort=False)
    df_dict[i]['molecule'] = df_dict[i]['molecule'].fillna(ctr_cmpd)
    i+=1
# Display a sample of the assembled DataFrame
display_markdown('### This is a sample of the imported data for plate ' + plate_name[0]+':',raw=True)
df_dict[0].head(5)

***
### Normalize data
Perform a normalization of the data against the positive control data.

In [ ]:
ctr_stat_list = []
ctr_stat_dict = {}
meanval = '{}_MEAN'
stdval = '{}_STD_DEV'
featurecol = df_dict[0].columns
featurecol = featurecol[3:len(featurecol)]
ctr_compound_list = [ctr_cmpd,'4','782','951','958','990','C0.3','C1','C3','T0.3','T1','T3'] # WOrk to set this as a future input text file

for ctr_compound in ctr_compound_list:
    for plate in df_dict:
        stat=OrderedDict()
        wellpos=[]
        ctr_compound_df = df_dict[plate].loc[df_dict[plate]['molecule']==ctr_compound]
        stat['Plate'] = plate_name[plate]
        stat['Compound'] = ctr_compound
        for feature in featurecol:
            stat[meanval.format(feature)] = ctr_compound_df[feature].mean()
            stat[stdval.format(feature)] = ctr_compound_df[feature].std()
        for row in ctr_compound_df.itertuples():
            wellpos.append('{}{}'.format(chr(row.pRow+64),row.pCol))
        stat['Count'] = ctr_compound_df['molecule'].count()
        stat['WellPosition'] = wellpos
        ctr_stat_list.append(stat)
ctr_stat_df = pd.DataFrame(ctr_stat_list)

# df_norm is a dictionary of normalized data from the dictionary df_dict
df_norm=df_dict
ctr_cmpd_df=ctr_stat_df.loc[ctr_stat_df['Compound']==ctr_cmpd]
for plate in df_norm:
    df_temp = df_norm[plate]
    for feature in featurecol:
        normval = ctr_cmpd_df.loc[ctr_cmpd_df['Plate'] == plate_name[plate], feature+'_MEAN'].tolist()[0]
        df_temp[feature] = 100*(1-df_temp[feature]/normval)
            
    df_norm.update({plate: df_temp})

display(ctr_stat_df.head())

***
### Display Data

In [ ]:
def update_heatmap(plate_norm,feature_norm,table,plate_name):
    #Fetch the data corresponding to the sliders
    df = df_dict[plate_norm]
    col_name = df.columns[feature_norm+3]
    
    #Display Heatmap
    trace_heatmap = go.Heatmap(
    x=df['pCol'],
    y=df['pRow'],
    z=df[col_name],
    text = df['molecule'].tolist(),
    hoverinfo = 'text+x+y+z',
        )
    
    data = [trace_heatmap]
    layout_heatmap = dict(
        title = 'Plate: {} Feature: {}'.format(plate_name[plate_norm],featurecol[feature_norm]),
        xaxis = dict(
            title = 'Column',
            tickmode = 'array',
            tickvals = df['pCol'].unique()
        ),
        yaxis = dict(
            title = 'Row',
            tickmode = 'array',
            tickvals = df['pRow'].unique(),
            ticktext = list(string.ascii_uppercase),
            autorange='reversed'
        ),
              )
    fig = go.Figure(data=data, layout=layout_heatmap)
    display(py.iplot(fig))
    #plot(fig, filename = 'figure_1.html')
    #display(HTML('figure_1.html'))
    return

def update_candidates(plate_norm,feature_norm,thresh_multiplier,celldeath_thresh,plate_name,df_norm,ctr_cmpd_df,featurecol):
    
    # Display control compound mean and standard dev for plate and feature and place it in candidates tab
    avgfeature = ctr_cmpd_df.loc[ctr_cmpd_df['Plate'] == plate_name[plate_norm], featurecol[feature_norm]+'_MEAN'].tolist()[0]
    stdfeature = ctr_cmpd_df.loc[ctr_cmpd_df['Plate'] == plate_name[plate_norm], featurecol[feature_norm]+'_STD_DEV'].tolist()[0]
    avgcell = ctr_cmpd_df.loc[ctr_cmpd_df['Plate'] == plate_name[plate_norm], 'ValidObjectCount_MEAN'].tolist()[0]
    stdcell = ctr_cmpd_df.loc[ctr_cmpd_df['Plate'] == plate_name[plate_norm], 'ValidObjectCount_STD_DEV'].tolist()[0]
    
    display_markdown('<h5>{} Control Value<br></h5>Plate #: {}<br>{}: {}+-{}<br>CellCount: {} +- {}'.format(
        ctr_cmpd,
        plate_name[plate_norm],
        featurecol[feature_norm],
        int(avgfeature),
        int(stdfeature),
        int(avgcell),
        int(stdcell)),
        raw=True)
    
    # Display list of candidates that satisfy criteria for cell number and feature
    fractSTD = stdfeature/avgfeature
    feature_threshold = 100*thresh_multiplier*fractSTD
    hitslist = df_norm[plate_norm][['molecule',featurecol[feature_norm],'ValidObjectCount']].loc[(df_norm[plate_norm][featurecol[feature_norm]] >= (feature_threshold)) & (df_norm[0]['ValidObjectCount'] <= (celldeath_thresh))]

    display_markdown('<h5>Number of candidates: {}</h5>'.format(len(hitslist)), raw=True)
    display(hitslist)
    
    return hitslist

def update_waterfall(plate_norm,feature_norm,plate_name,df_norm,ctr_cmpd_df,featurecol):
    
    # Display control compound mean and standard dev for plate and feature and place at the top of the waterfall tab
    avgfeature = ctr_cmpd_df.loc[ctr_cmpd_df['Plate'] == plate_name[plate_norm], featurecol[feature_norm]+'_MEAN'].tolist()[0]
    stdfeature = ctr_cmpd_df.loc[ctr_cmpd_df['Plate'] == plate_name[plate_norm], featurecol[feature_norm]+'_STD_DEV'].tolist()[0]
    avgcell = ctr_cmpd_df.loc[ctr_cmpd_df['Plate'] == plate_name[plate_norm], 'ValidObjectCount_MEAN'].tolist()[0]
    stdcell = ctr_cmpd_df.loc[ctr_cmpd_df['Plate'] == plate_name[plate_norm], 'ValidObjectCount_STD_DEV'].tolist()[0]
    
    print('plate #:',plate_name[plate_norm], '\n'+ctr_cmpd+' Control Value:', featurecol[feature_norm]+':', int(avgfeature), '+-', int(stdfeature), 'CellCount:', int(avgcell), '+-', int(stdcell))
    
    #Display waterfall plot
    test_df_norm=df_norm[plate_norm].sort_values(by=featurecol[feature_norm])
    xval = list(range(1, len(test_df_norm.index)))
    yval_feature = test_df_norm[featurecol[feature_norm]]
    yval_viability = test_df_norm['ValidObjectCount']
    molecule_text = test_df_norm['molecule'].tolist()
    xval_control = [i for i, x in enumerate(molecule_text,1) if x == ctr_cmpd]
    yval_feature_control = test_df_norm[featurecol[feature_norm]].loc[test_df_norm['molecule']=='DMSO']    

    trace_feature = go.Scatter(
        x=xval,
        y=yval_feature,
        marker = dict(
            size = 5,
            color = 'rgba(152, 0, 0, .8)',
        ),
        mode="markers",
        text=molecule_text,
        name='Normalized '+featurecol[feature_norm]
    )
    
    trace_feature_control = go.Scatter(
        x=xval_control,
        y=yval_feature_control,
        marker = dict(
            size = 5,
            color = 'rgba(152, 0, 0, .8)',
            line = dict(
                width = 1,
                color = 'rgb(0,0,0)'
            )
        ),
        mode="markers",
        hoverinfo = 'skip',
        name='Normalized '+ctr_cmpd
    )

    trace_viability = go.Scatter(
        x=xval,
        y=yval_viability,
        marker = dict(
            size = 5,
            color = 'rgba(0, 0, 255, .2)',
        ),
         mode="markers",
         name='Normalized Cell Viability'
    )
    
    data_waterfall=[trace_feature, trace_viability,  trace_feature_control]
    layout_waterfall = dict(
        title = 'Plate: {} Feature: {}'.format(plate_name[plate_norm],featurecol[feature_norm] ),
        yaxis = dict(title = 'Normalized Value'),
              )
    figure_waterfall = dict(data=data_waterfall, layout=layout_waterfall)
    display(py.iplot(figure_waterfall))
    
    #plot(figure_waterfall, filename = 'figure_2.html')
    #display(HTML('figure_2.html'))
    return

plate_slider = widgets.IntSlider(
    min=0,
    max=(len(plate_name)-1),
    description='Plate:'
    )

feature_slider = widgets.IntSlider(
    min=0,
    max=(len(df_dict[0].columns)-4),
    description='Feature:'
    )

celldeath_thresh = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    description='% cell death thresh:'
    )

thresh_multiplier = widgets.FloatSlider(
    value=3,
    min=0,
    max=10,
    description='%Std.Dev multiplier:'
    )

export_candidates_button = widgets.Button(
    description='Export Candidates List',
    )

export_stats_button = widgets.Button(
    description='Export Stats',
    )

export_norm_data_button = widgets.Button(
    description='Export Normalized Data',
    )

plate_heatmap = widgets.interactive(
    update_heatmap,
    plate_norm=plate_slider,
    feature_norm=feature_slider,
    table=fixed(df_dict),
    plate_name=fixed(plate_name)
    )

plate_waterfall = widgets.interactive(
    update_waterfall,
    plate_norm=plate_slider,
    feature_norm=feature_slider,
    plate_name=fixed(plate_name),
    df_norm=fixed(df_norm),
    ctr_cmpd_df=fixed(ctr_cmpd_df),
    featurecol =fixed(featurecol)
    )

plate_candidates = widgets.interactive(
    update_candidates,
    plate_norm=plate_slider,
    feature_norm=feature_slider,
    thresh_multiplier=thresh_multiplier,
    celldeath_thresh=celldeath_thresh,
    plate_name=fixed(plate_name),
    df_norm=fixed(df_norm),
    ctr_cmpd_df=fixed(ctr_cmpd_df),
    featurecol =fixed(featurecol)
    )

full_waterfall = widgets.Output()
with full_waterfall:
    fulldf = pd.DataFrame()
    for plate in df_norm:
        fulldf = pd.concat(df_norm) 
    
    l = fulldf[['molecule',featurecol[0],'ValidObjectCount']].values.tolist()
    l.sort(key=lambda x: x[1])
    xvalFull = list(range(1, len(fulldf['molecule'])))
    yvalFeature = []
    yvalText = []
    yvalObject = []
    yvalFeatureCtr = []
    yvalObjectCtr = []
    xvalCtr = []
    i=0
    for sublist in l:
        i+=1
        yvalFeature.append(sublist[1])
        yvalText.append(sublist[0])
        yvalObject.append(sublist[2])
        if sublist[0] == ctr_cmpd:
            xvalCtr.append(i)
            yvalFeatureCtr.append(sublist[1])
            yvalObjectCtr.append(sublist[2])
    
    trace_featureFull = go.Scatter(
        x=xvalFull,
        y=yvalFeature,
        marker = dict(
            size = 5,
            color = 'rgba(152, 0, 0, .8)',
        ),
        mode="markers",
        text=yvalText,
        name='Normalized Feature'
    )

    trace_viabilityFull = go.Scatter(
        x=xvalFull,
        y=yvalObject,
        marker = dict(
            size = 5,
            color = 'rgba(0, 0, 255, .2)',
        ),
         mode="markers",
         name='Normalized Cell Viability'
    )
    
    trace_ctr = go.Scatter(
        x=xvalCtr,
        y=yvalFeatureCtr,
        marker = dict(
            size = 5,
            color = 'rgba(152, 0, 0, 0.8)',
            line = dict(
                width = 1,
                color = 'rgb(0,0,0)'
            )
        ),
         mode="markers",
         name='Normalized {} Values'.format(ctr_cmpd)
    )
    
    data_waterfallFull=[trace_featureFull, trace_viabilityFull, trace_ctr]
    layout_waterfallFull = dict(
        title = 'Full Waterfall',
        yaxis = dict(title = 'Normalized Value'),
              )
    figure_waterfallFull = dict(data=data_waterfallFull, layout=layout_waterfallFull)
    display(py.iplot(figure_waterfallFull))
#     plot(figure_waterfallFull, filename = 'figure_3.html')
#     display(HTML('figure_3.html'))
        

show_table = widgets.Output()
with show_table:
    display(ctr_stat_df)

def export_norm_data(b=None):    
    excelpath = r'MoleculeData/output'
    writer = pd.ExcelWriter(excelpath + '/test_norm_data.xlsx', engine='xlsxwriter')
    # Write normalized dataframe to excel
    for i in df_norm:
        df_norm[i].to_excel(writer, sheet_name=plate_name[i], index_label='WellID')
    writer.save()
    
def export_candidates_list(b=None):    
    excelpath = r'MoleculeData/output'
    writer = pd.ExcelWriter(excelpath + '/test_candidates_list.xlsx', engine='xlsxwriter')
    # Write hitslist to first sheet of excel
    plate_candidates.result.to_excel(writer, sheet_name='HitsList', index_label='WellID')
    writer.save()
    
def export_stats_list(b=None):    
    excelpath = r'MoleculeData/output'
    writer = pd.ExcelWriter(excelpath + '/test_stats.xlsx', engine='xlsxwriter')
    # Write control stats to first sheet of excel
    ctr_stat_df.to_excel(writer, sheet_name='Control_stats', index_label='WellID')
    writer.save()

@export_norm_data_button.on_click
def export_norm_data_on_click(b):
    export_norm_data()
    
@export_candidates_button.on_click
def export_candidates_on_click(b):
    export_candidates_list()
    
@export_stats_button.on_click
def export_stats_on_click(b):
    export_stats_list()

In [ ]:
tab1 = VBox(children=[plate_heatmap
                     ])
tab2 = VBox(children=[plate_waterfall
                     ])
tab3 = VBox(children=[export_candidates_button,
                      plate_candidates
                     ])
tab4 = VBox(children=[export_stats_button,
                      show_table
                     ])

tab = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tab.set_title(0, 'Heatmap')
tab.set_title(1, 'Waterfall per Plate')
tab.set_title(2, 'Candidate List')
tab.set_title(3, 'Statistics')
VBox(children=[ full_waterfall, export_norm_data_button, tab])